In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pylab as plt
%matplotlib inline

In [23]:
from astroquery.simbad import Simbad
from astropy.table import Table, vstack

def read_simbad_table(obj_name, **kwargs):
    """Read in Astroquery results from Simbad given object name (str)
    
    Parameter
    ---------
    obj_name : (str) name of object
    
    kwargs (optional)
    ------
    votfields : (list) list of votable field keywords, from http://simbad.u-strasbg.fr/Pages/guide/sim-fscript.htx
                default is ['sptype', 'fluxdata(V)', 'velocity', 'distance']
    """
    customSimbad = Simbad()
    votfields = kwargs.pop('votfields', ['sptype', 'fluxdata(V)', 'velocity', 'distance'])
#    customSimbad.add_votable_fields('sptype')
#    customSimbad.add_votable_fields('fluxdata(V)')
#    customSimbad.add_votable_fields('velocity')
#    customSimbad.add_votable_fields('distance')
    for vf in votfields:
        customSimbad.add_votable_fields(vf)
    result_table = customSimbad.query_object(obj_name)
    return result_table

TWOPI = 2*np.pi
def get_angular_separation(ra1, dec1, ra2, dec2):
    """Compute the angular separation in radians between 2 objects given positions in degrees.
    
    Parameters
    ----------
    ra1: (float) right ascension of object 1 in degrees
    dec1: (float) declination of object 1 in degrees
    ra2: (float) right ascension of object 2 in degrees
    dec2: (float) declination of object 2 in degrees"""
    assert ra1>=0. and ra1<=TWOPI and ra2>=0. and ra2<=TWOPI, "RA must be in [0, 2pi]: {} {}".format(ra1, ra2)
    assert dec1>=-np.pi/2. and dec1<=np.pi/2. and dec2>=-np.pi/2. and dec2<=np.pi/2., "DEC must be in [-pi/2, pi/2]: {} {}".format(dec1, dec2)
    cosA = np.sin(dec1)*np.sin(dec2) + np.cos(dec1)*np.cos(dec2)*np.cos(ra1-ra2)
    return np.arccos(cosA)

In [ ]:
## read in the biased draw from Aki and make 5 unbiased draws from them
Ndraw=5
obj_names_all = np.loadtxt('inputs/luvoir-A_stars.txt', usecols=(0,), delimiter=',', skiprows=1).astype(int)
obj_names = np.random.choice(obj_names_all, size=Ndraw, replace=False)
print(obj_names)

## stack each Simbad query results vertically in Astropy Table
result = read_simbad_table("Hip"+str(obj_names[0]))
for ii in range(1,len(obj_names)):
    result = vstack([result, read_simbad_table("Hip"+str(obj_names[ii]))])
    
print(result)

[99461 56997 82588 14632 95447]


In [18]:
result['MAIN_ID', 'SP_TYPE', 'FLUX_V', 'FLUX_ERROR_V', 'RVZ_RADVEL', 'Distance_distance']

MAIN_ID,SP_TYPE,FLUX_V,FLUX_ERROR_V,RVZ_RADVEL,Distance_distance
,,mag,,km / s,
object,object,float32,float32,float64,float64
HD 50281,K3.5V,6.57,--,-6.910,8.7492
* 22 Lyn,F6V,5.357,0.009,-27.330,20.3962
* zet02 Ret,G1V,5.24,--,12.010,12.0464
* bet Vir,F9V,3.6,--,4.710,11.1203
* zet Dor,F9VFe-0.5,4.708,0.009,-1.150,11.6247


In [4]:
## convert the Simbad RA/DEC from hh.mm.ss to degrees
from astropy import units as u
from astropy.coordinates import SkyCoord

c = SkyCoord(result['RA'], result['DEC'], unit=(u.hourangle, u.deg))

In [5]:
angular_separations = []
for ii in range(len(obj_names)-1):
    angsep = get_angular_separation(c.ra[ii].radian, c.dec[ii].radian, 
                                    c.ra[ii+1].radian, c.dec[ii+1].radian) * 180/np.pi
    angular_separations.append(angsep)
    print("Ang sep [deg] btw. HIP {} and {} = {}".format(obj_names[ii], obj_names[ii+1], 
                                           angsep))

Ang sep [deg] btw. HIP 77760 and 19335 = 99.4271924513
Ang sep [deg] btw. HIP 19335 and 47592 = 99.6220061544
Ang sep [deg] btw. HIP 47592 and 43726 = 21.7838760231
Ang sep [deg] btw. HIP 43726 and 3093 = 123.337959426


In [66]:
# to compute the slew time need to know how the instrument will slew:
# take shortest path vs. along 1 axis then another...?

def get_slew_times(sep, vel=10.):
    return

In [ ]:
# fn to compute the astrometric (proper motion + parallax) signal given a set of parameters

# fn to parse the set of parameters for fiducial targets

# fn to compute the integration time/epochs needed for astrometric signal

# fn to compute slew time

# fn to minimize slew time



In [6]:
from astroquery.vizier import Vizier
result = Vizier.query_object("HIP"+str(obj_names[0]))

In [13]:
Simbad.query_objects?

TypeError: cannot concatenate 'str' and 'list' objects